<a href="https://colab.research.google.com/github/owenbean400/AmazonSentimentAnalysis/blob/master/Anne_Turmel_Owen_Bean_Amazon_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Anne Turmel and Owen Bean NLP Final Project Spring 2023

Placeholder for project summary; write when writing abstract for report? 

In [35]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
# imports
import os
import json
import gzip
import csv
import pandas as pd
from urllib.request import urlopen
# Data visualization
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='whitegrid')
import torch
torch.manual_seed(1324224321)
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import numpy as np
from datasets import load_metric
import scipy.stats

Read in file data

In [38]:
file_name = "Movies_and_TV.csv"
data = pd.read_csv(file_name)
data.head()

,reviewerID,overall,reviewText_LUKE,reviewText
0,AHTYUW2H1276L,5.0,This movie was in <NNP>....it was a great summ...,This movie was in ENGLISH....it was a great su...
1,A1OMHX76O2NC6V,1.0,This DVD appears to be in <NNP>. It is not in ...,This DVD appears to be in German. It is not in...
2,A3603DM2201OZP,5.0,"Excellent dvd, lots of fun and great songs! <...","Excellent dvd, lots of fun and great songs! S..."
3,A3HE4QW1655VB9,5.0,My grandkids loved this and it made it easy fo...,My grandkids loved this and it made it easy fo...
4,A1CLLD4H7TYKKJ,4.0,I'm happy with my purchase,I'm happy with my purchase


Preliminary data exploration

In [39]:
data.shape

(19942, 4)

In [40]:
# metadata 
feat_desc = pd.DataFrame({'Description': ['ID of person who wrote the review',
                                          'Rating review gives to item',
                                          'Text of review with proper nouns masked',
                                          'Text of review unparsed'], 
                          'Values': [data[i].unique() for i in data.columns],
                          'Number of unique values': [len(data[i].unique()) for i in data.columns]}, 
                          index = data.columns)
feat_desc

,Description,Values,Number of unique values
reviewerID,ID of person who wrote the review,"[AHTYUW2H1276L, A1OMHX76O2NC6V, A3603DM2201OZP...",19544
overall,Rating review gives to item,"[5.0, 1.0, 4.0, 3.0, 2.0]",5
reviewText_LUKE,Text of review with proper nouns masked,[This movie was in <NNP>....it was a great sum...,16825
reviewText,Text of review unparsed,[This movie was in ENGLISH....it was a great s...,16919


In [41]:
data.dtypes

reviewerID          object
overall            float64
reviewText_LUKE     object
reviewText          object
dtype: object

In [42]:
# Find missing data
pd.DataFrame({'Number of Missing Values': data.isna().sum(), 
              '% of Missing Values': (data.isna().sum()/data.shape[0] * 100).round(2)})

,Number of Missing Values,% of Missing Values
reviewerID,0,0.00
overall,0,0.00
reviewText_LUKE,1,0.01
reviewText,1,0.01


In [43]:
# establish the same model for original text and LUKE text
model_name = "albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5).to(device)
model_luke = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5).to(device)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.bias', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

In [44]:
# reads review information into dictionaries
# separates original text from LUKE text
def read_amazon_product_review(file_name):
    df = pd.read_csv(file_name, sep=",", header=0)
    dic_review = {}
    dic_review_LUKE = {}
    df1 = df[["reviewerID", "overall", "reviewText_LUKE", "reviewText"]].dropna()
    #print(df.shape)
    for index in range(len(df1)):
        rating = df.loc[index, "overall"]
        label = int(rating) - 1
        
        review_id = df.loc[index, "reviewerID"]
        if label in dic_review:
            dic_review[label][review_id] = df.loc[index, "reviewText"]
        else:
            dic_review[label] = {review_id: df.loc[index, "reviewText"]}
        if label in dic_review_LUKE:
            dic_review_LUKE[label][review_id] = df.loc[index, "reviewText_LUKE"]
        else:
            dic_review_LUKE[label] = {review_id: df.loc[index, "reviewText_LUKE"]}

    return dic_review, dic_review_LUKE

In [45]:
# splits data 80-10-10 for training, validation, and test
def split_data(amazon_data):
    training = {}
    validation = {}
    test = {}

    for label in amazon_data:
        temp_dic = amazon_data[label]
        lst_amazon_ids = list(temp_dic.keys())
        train_length = int(len(lst_amazon_ids) * 0.8)
        train_ids = lst_amazon_ids[:train_length]
        remaining = lst_amazon_ids[train_length:]
        test_length = int(len(remaining) * 0.5)
        test_ids = remaining[:test_length]
        validation_id = remaining[test_length:]

        for id in train_ids:
            training[temp_dic[id]] = label
        for id in validation_id:
            validation[temp_dic[id]] = label
        for id in test_ids:
            test[temp_dic[id]] = label

    print(test)

    return training, validation, test

In [50]:
dic_review, dic_review_LUKE = read_amazon_product_review("all_beauty.csv")
training, validation, test = split_data(dic_review_LUKE)
training_luke, validation_luke, test_luke = split_data(dic_review_LUKE)

{'After only a couple uses the barrel disconnected from the handle and moved freely around.  Basically a waste of money': 0, 'Not happy!!!! Second time I tried to use it a row of the brush came out!  Would not recommend this prouduct': 0, 'Took it from the package plugged it in but it never worked. Disappointing..Excited to find one this size but it never got hot.': 0, 'This product is deffective.  The end turns and the bristles will come out and I would really like to know how to send it back and get a replacement.': 0, 'I am sorry I bought this product. Takes 15 minutes to heat up. Who has that kind of time? Save your money.': 0, "It doesn't get hot enough and tangles in even short hair. There is no curl effect as it does not get hot.": 0, 'It fell apart as soon as I took it out of the box. I could not use it.  I returned it.': 0, 'Very low heat at max, not effective for my short thick hair.  Tangles easily in hair. I should have returned it': 0, 'This was very cheap feeling, did not

In [47]:
class AmazonProductReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [51]:
# get encodings to train with
train_encodings = tokenizer(list(training.keys()), truncation=True, padding=True)
val_encodings = tokenizer(list(validation.keys()), truncation=True, padding=True)
test_encodings = tokenizer(list(test.keys()), truncation=True, padding=True)
#train_encodings_luke = tokenizer(list(training_luke.keys()), truncation=True, padding=True)
#val_encodings_luke = tokenizer(list(validation_luke.keys()), truncation=True, padding=True)
#test_encodings_luke = tokenizer(list(test_luke.keys()), truncation=True, padding=True)
train_dataset = AmazonProductReviewDataset(train_encodings, list(training.values()))
val_dataset = AmazonProductReviewDataset(val_encodings, list(validation.values()))
test_dataset = AmazonProductReviewDataset(test_encodings, list(test.values()))
#train_dataset_luke = AmazonProductReviewDataset(train_encodings_luke, list(training.values()))
#val_dataset_luke = AmazonProductReviewDataset(val_encodings_luke, list(validation.values()))
#test_dataset_luke = AmazonProductReviewDataset(test_encodings_luke, list(test.values()))

In [54]:
model_name = "fine_tune_amazon"
model_name_luke = "fine_tune_amazon_LUKE"
learning_rate = 2e-5
per_device_train_batch_size = 16
per_device_eval_batch_size = 16
num_train_epochs = 5
save_strategy = "no"

In [55]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   return {"accuracy": accuracy}

In [56]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
training_args = TrainingArguments(
   output_dir=model_name,
   learning_rate=learning_rate,
   per_device_train_batch_size=per_device_train_batch_size,
   per_device_eval_batch_size=per_device_eval_batch_size,
   num_train_epochs=num_train_epochs,
   save_strategy = save_strategy
)

In [57]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=val_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [58]:
training_args_luke = TrainingArguments(
   output_dir=model_name_luke,
   learning_rate=learning_rate,
   per_device_train_batch_size=per_device_train_batch_size,
   per_device_eval_batch_size=per_device_eval_batch_size,
   num_train_epochs=num_train_epochs,
   save_strategy = save_strategy
)

In [60]:
#trainer_luke = Trainer(
 #  model=model_luke,
  # args=training_args,
   #train_dataset=train_dataset_luke,
   #eval_dataset=val_dataset_luke,
   #tokenizer=tokenizer,
   #data_collator=data_collator,
   #compute_metrics=compute_metrics,
#)

In [61]:
# train
print("Training one")
trainer.train()
trainer.save_model()
print(trainer.evaluate())
# Test dataset
trainer.eval_dataset = test_dataset

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Training one


Step,Training Loss


KeyboardInterrupt: ignored

In [ ]:
# test
print("Testing one")
print(trainer.evaluate())
# or
trainer.predict(test_dataset)

In [ ]:
# perform t-test on both results to see if the difference is significant
scipy.stats.ttest_ind(test, test_LUKE, equal_var=True)